In [1]:
import nbimporter
from augmentation import *
from parsing import *
from subsampling import *
import os
from collections import Counter
import nltk

In [2]:
PATH = os.getcwd() + '/tar-project/'
PATIENT_NOTES_PATH = PATH + 'patient_notes.csv'
FEATS_PATH = PATH + 'features.csv'
TRAIN_PATH = PATH + 'train.csv'
TEST_PATH = PATH + 'test.csv'
VALID_PATH = PATH + 'valid.csv'


df_patient_notes = pd.read_csv(PATIENT_NOTES_PATH)
df_features = pd.read_csv(FEATS_PATH)
df_train = pd.read_csv(TRAIN_PATH)
df_test = pd.read_csv(TEST_PATH)
df_valid = pd.read_csv(VALID_PATH)

In [3]:
def count(row):
    return row[2].split('OR')

df_features = df_features.loc[df_features['case_num'].isin([0, 1, 2, 3, 4])]
df_features['num_of_features'] = df_features.apply(count, axis=1)

s = set()
for l in df_features['num_of_features'].tolist():
    for feature in l:
        s.add(feature.lower())
print('Unique features: ', end='')
print(len(s))

Unique features: 84


In [4]:
def get_num_all_features(df):
    
    def count(row):
        if row[5] == '[]':
            return 0
        return len(row[5].split(','))
    
    df = df.loc[df['case_num'].isin([0, 1, 2, 3, 4])]
    df['num_of_location'] = df.apply(count, axis=1)
    
    return df['num_of_location'].sum()


In [5]:
print('All features train: ', end='')
print(get_num_all_features(df_train))
print('All features test: ', end='')
print(get_num_all_features(df_test))
print('All features valid: ', end='')
print(get_num_all_features(df_valid))

All features train: 2347
All features test: 1284
All features valid: 1820


In [6]:
def get_all_tokens(df):
    
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    df_pn = df_patient_notes.loc[df_patient_notes['pn_num'].isin(df['pn_num'].tolist())]
    sentences = df_pn['pn_history'].tolist()
    instances = [tokenizer.tokenize(sentence.lower()) for sentence in sentences]
    sentences_clean = []
    
    for sentence in instances:
        for word in sentence:
            sentences_clean.append(word.lower())
    freq = dict(Counter(sentences_clean))

    
    return sum(freq.values()), len(freq)

In [7]:
a, u = get_all_tokens(df_train)
print('All tokens train: {} \nUnique tokens train: {}\n'.format(a, u))
a, u = get_all_tokens(df_test)
print('All tokens test: {} \nUnique tokens test: {}\n'.format(a, u))
a, u = get_all_tokens(df_valid)
print('All tokens valid: {} \nUnique tokens valid: {}\n'.format(a, u))

All tokens train: 27182 
Unique tokens train: 2383

All tokens test: 14259 
Unique tokens test: 1708

All tokens valid: 21008 
Unique tokens valid: 2062

